In [2]:
!pip list | grep -E "(adk|genai|aiplatform)"

google-adk                              1.5.0
google-cloud-aiplatform                 1.101.0
google-genai                            1.21.1


In [5]:
import copy, datetime, json, os, pprint, time, uuid
import vertexai
from google.genai.types import Part, Content
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import google_search
from google.adk.tools.agent_tool import AgentTool

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'
vertexai.init(
    project=PROJECT_ID, location=LOCATION,
    staging_bucket=f'gs://{PROJECT_ID}'
)
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [4]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print(f'===\n{event}\n===')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

In [11]:
google_search_tool = LlmAgent(
    model='gemini-2.0-flash',
    name='google_search_tool',
    instruction="""
    You are a Google Search expert.
    """,
    tools=[google_search],
)

instruction = '''
You are a friendly AI assistant that answers user's queries.

Step1. Use google_search_tool to get the latest information.
Step2. Give answers based on the latest and objective information.

[Format instruction]
Output in English, in plain text only.
Avoid adding citation marks such as [1][2].
'''

# With skip_summarization=False
search_agent1 = LlmAgent(
    name='search_agent1',
    model='gemini-2.0-flash-001',
    description='Agent to answer questions using Google Search.',
    instruction=instruction,
    tools=[AgentTool(google_search_tool, skip_summarization=False)],
)

# With skip_summarization=True
search_agent2 = LlmAgent(
    name='search_agent2',
    model='gemini-2.0-flash-001',
    description='Agent to answer questions using Google Search.',
    instruction=instruction,
    tools=[AgentTool(google_search_tool, skip_summarization=True)],
)

With `skip_summarization=False`, the agent generate a response based on the `google_search_tool`'s result.

In [8]:
client = LocalApp(search_agent1)
DEBUG=True
query = '''
Get the wheather news in Seoul.
'''
_ = await client.stream(query)

===
content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-63605f31-c3b5-45dc-b065-ac2e3f2aaf0e', args={'request': 'weather in Seoul'}, name='google_search_tool'), function_response=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=9, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=9)], prompt_token_count=121, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=121)], thoughts_token_count=None, tool_use_prompt_token_count=None, tool_use_prompt_tokens_details=None, total_token_count=130, traffic_type=<Traffi

With `skip_summarization=True`, the agent stops working when it receives a function call response from the `google_search_tool`'s result. **It doesn't generate any text reposonce.** So the application (calling the agent) needs to parse the funcion call response event by itself.

In [9]:
client = LocalApp(search_agent2)
DEBUG=True
query = '''
Get the wheather news in Seoul.
'''
_ = await client.stream(query)

===
content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-c54150ba-60b4-4c5c-a089-e19eeefcb992', args={'request': 'weather in Seoul'}, name='google_search_tool'), function_response=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=9, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=9)], prompt_token_count=121, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=121)], thoughts_token_count=None, tool_use_prompt_token_count=None, tool_use_prompt_tokens_details=None, total_token_count=130, traffic_type=<Traffi